In [1]:
#importing the required libraries
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv
/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv


In [2]:
#reading the dataset
dfSub = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/sample_submission.csv')
dfStarter = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/census_starter.csv')
dfTrain = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')
dfTest = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/test.csv')

In [3]:
#Making a Copy of The Dataset
#I would be working with this copy through out the project while the original dataframe would be for referencing when needed .
df_Sub = dfSub.copy()
df_Starter = dfStarter.copy()
df_Train = dfTrain.copy()
df_Test =dfTest.copy()

In [4]:
print(df_Train.shape)
print(df_Test.shape)
print(df_Sub.shape)
print(df_Starter.shape)

(122265, 7)
(25080, 3)
(25080, 2)
(3142, 26)


In [5]:
#information on the test dataset
df_Test.head()

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01


In [6]:
#information on train datset
df_Train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [7]:
df_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [8]:
len(df_Test), len(df_Train)

(25080, 122265)

## Data Cleaning

In [9]:
def get_first_day_of_month(item):
    item = item.split('_')[1]
    return item

def get_cfips(item):
    item = item.split('_')[0]
    return item

In [10]:
df_Sub['first_day_of_month'] = df_Sub['row_id'].apply(get_first_day_of_month)
df_Sub['cfips'] = df_Sub['row_id'].apply(get_cfips)

In [11]:
#converting the date to datetime
df_Test['first_day_of_month'] = pd.to_datetime(df_Test['first_day_of_month'])
df_Train['first_day_of_month'] = pd.to_datetime(df_Train['first_day_of_month'])
df_Sub['first_day_of_month'] = pd.to_datetime(df_Sub['first_day_of_month'])

In [12]:
#dropping the `row_id` column
df_Test.drop(['row_id'],axis = 1, inplace = True)
df_Train.drop(['row_id'],axis = 1, inplace = True)
df_Sub.drop(['row_id'],axis = 1, inplace = True)

In [13]:
#merging the dataset 
df_trainMerged = df_Train.merge(df_Starter, on = 'cfips', how = 'left')
df_trainMerged

,cfips,county,state,first_day_of_month,microbusiness_density,active,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
2,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
3,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
4,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,76.6,78.9,80.6,82.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122260,56045,Weston County,Wyoming,2022-06-01,1.803249,101,71.1,73.3,76.8,79.7,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122261,56045,Weston County,Wyoming,2022-07-01,1.803249,101,71.1,73.3,76.8,79.7,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122262,56045,Weston County,Wyoming,2022-08-01,1.785395,100,71.1,73.3,76.8,79.7,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0
122263,56045,Weston County,Wyoming,2022-09-01,1.785395,100,71.1,73.3,76.8,79.7,...,0.6,0.6,0.0,0.0,0.0,59605,52867.0,57031,53333.0,65566.0


In [14]:
#confirming the null values in the data,this shows there are no null values in the dataset.
df_trainMerged.isnull().sum()

cfips                     0
county                    0
state                     0
first_day_of_month        0
microbusiness_density     0
active                    0
pct_bb_2017               0
pct_bb_2018               0
pct_bb_2019               0
pct_bb_2020               0
pct_bb_2021               0
pct_college_2017          0
pct_college_2018          0
pct_college_2019          0
pct_college_2020          0
pct_college_2021          0
pct_foreign_born_2017     0
pct_foreign_born_2018     0
pct_foreign_born_2019     0
pct_foreign_born_2020     0
pct_foreign_born_2021     0
pct_it_workers_2017       0
pct_it_workers_2018      39
pct_it_workers_2019       0
pct_it_workers_2020       0
pct_it_workers_2021       0
median_hh_inc_2017        0
median_hh_inc_2018       39
median_hh_inc_2019        0
median_hh_inc_2020       39
median_hh_inc_2021       39
dtype: int64